# Blog: Leveraging the Landsat Time Series to Understand Development Surrounding Denver International Airport (DIA)
By: Richard Udell (July 2020)

# Introduction
I sat in a 25 square foot room in Kansas for 2,500 hours and was paid for it!  Well, it wasn’t actually Kansas, it was the plains 25 miles east of Denver, Colorado.  But if you’ve been there, you would think the landscape looks pretty similar (save the awesome view of the Rockies!).  Out there construction is happening at a dizzying rate as the Denver Metro area is replacing miles and miles of plains with homes, highways, and shopping centers.  My job for the past year was to help make sure that the millions of tonnes of Earth beneath is strong enough to support the rapidly growing city.  That’s how I found myself sitting inside the 25 square foot cab of a work truck for hours on end.  I was watching the Earth be transformed from plains to city and I started to get curious about how this change was happening at a regional level.

# Relevance
When the Denver International Airport (DIA) finished construction 25 years ago it stood isolated in the plains, 30 miles from the City of Denver.  However, as I have observed first-hand in the past year, DIA is no longer so far from the city.  In this project, I aim to understand the trends in urban development surrounding DIA.  

# Tools
This workflow is completed using the Google Earth Engine Python API, a powerful tool that integrates ["the most advanced cloud-based geospatial processing platform in the world"](https://developers.google.com/earth-engine#background) with the Python programming language - where there are many useful open-source packages for data analysis.


# Region of Interest
![Region of Interest](images/ROI_screenshot.png)
*Caption: The region of interest (ROI) is a 10 mile radius around the southwest corner of DIA.*

# Data
[Landsat 5](https://www.usgs.gov/land-resources/nli/landsat/landsat-5?qt-science_support_page_related_con=0#qt-science_support_page_related_con) has been collecting 30m resolution satellite imagery of the whole Earth since 1984 - over a decade before the completion of DIA in 1995.  Unfortunately, Landsat 5 was decommissioned in 2012; therefore, this analysis measures trends from 1995 to 2012.  Despite this limited temporal range, there are still 938 Landsat scenes during that time that cover the ROI.  And as you can see upon close inspection of the plot below, many changes occurred on the landscape surrounding DIA during this time.

Use the floating box on the right-hand side to toggle the 1995 and 2011 RGB (Red Green Blue) layers to compare the differences in the landscape.  Feel free to zoom and pan to get a closer look.  

*Note: 2011 is used in this comparison because there was a censor malfunction on Landsat5 that prevented the collection of satellite imagery in the first half of 2012.* 

In [1]:
# Import necessary packages
import os
import numpy as np
import matplotlib.pyplot as plt

import rasterio as rio
import earthpy as et

import ee
import geemap

In [2]:
# Initiate first interactive map
Map1 = geemap.Map(center=(39.791424, -104.809968), zoom=10)
Map1.add_basemap('HYBRID')  # Add Google Map

In [3]:
# Set ROI as 10 mile (16,093 meters) radius around the intersection of
# Pena Blvd and E-470 (about the SW corner of the airport)
roi = ee.Geometry.Point([-104.746984, 39.834241]).buffer(16093)

# Add ROI to map
Map1.addLayer(roi, {}, 'ROI', opacity=0.5)

In [4]:
# Function to get cloud free summer composite for given year
def pre_processL5(year, ROI):
    """Filters the Landsat 5 ImageCollection to the given year and ROI and returns
    a cloud free clipped composite ee.Image object.
    
    Parameters
    ----------
    year : int
        Year to be used to filter Landsat 5 ImageCollection.
        
    ROI : ee.Geometry
        Any ee.Geometry object.
    

    Returns
    ------
    composite : ee.Image
        A 7 band ee.Image object derived from Landsat 5 that is filtered to the
        given year and clipped to the given ROI.
    
    """

    # Get raw Landsat5 data input for simple composite
    landsatCollection = ee.ImageCollection('LANDSAT/LT05/C01/T1')

    # Generate date strings for .filterDate()
    d1, d2 = str(year)+'-01-01', str(year)+'-12-31'  # Whole year
    # IMPORTANT NOTE: Landsat 5 for 2012 should only be available after June

    # Filter Landsat8 collection by date and by ROI
    collection = landsatCollection.filterDate(d1, d2)

    # Create cloud-free composite with default params and clip to ROI
    composite = ee.Algorithms.Landsat.simpleComposite(collection)

    return composite.clip(ROI)


# Landsat 5 cloud-free composite clipped to ROI (1995-2011)
imagesL5_dict = {}
for i in range(1995, 2012):
    imagesL5_dict[str(i)] = pre_processL5(year=i, ROI=roi)

#### Plot 1 Title: RGB Comparison of the ROI from 1995 to 2011

In [5]:
# Visualize images in RGB
visParams = {'bands': ['B3', 'B2', 'B1'], 'gain': [1.4, 1.4, 1.1]}

# Add L5 - 1995 RGB
Map1.addLayer(imagesL5_dict['1995'], visParams, '1995_RGB')

# Add L5 - 2011 RGB (2012 has an error)
Map1.addLayer(imagesL5_dict['2011'], visParams, '2011_RGB')

# Visualize the interactive map just above the airport
Map1.setCenter(-104.777782, 39.835136, 11)
Map1

Map(center=[39.835136, -104.777782], controls=(WidgetControl(options=['position'], widget=HBox(children=(Toggl…

**Plot 1 Caption:** There are some striking differences between the 1995 and 2011 RGB images.  The most obvious is the development of the old Stapleton airport - in 2011 is under development on its way to becoming the Stapleton neighborhood of Denver.  Additionally, a major change that occurred over the 16 years in the comparison is the development of Green Valley Range (first development southwest of DIA).

After viewing the RGB comparison plot, it is clear that there are changes in urban land cover surrounding DIA.  In the methods section I discuss how I went about measuring these changes.
# Methods
Spectral unmixing was used to determine the fraction of each pixel in the entire Landsat 5 collection (1995-2012) that is urban.  Then, the trend in urban over time was be measured by finding a linear line that best fits the change in urban fraction over time.  

Spectral unmixing can seem like magic, so I will attempt to explain (roughly) how it works.  Spectral unmixing can theoretically determine whether the land at each pixel is one of three categories: urban, vegetation, or water.  Each pixel in a Landsat image has a number that basically tells you how bright that small piece of Earth is.  Spectral unmixing considers how bright a piece of Earth should be if it is urban, vegetation, or water and calculates which category each pixel fits into.  However, it’s a bit more complicated - spectral unmixing assigns a percentage of each category to each pixel.  For example: if you examine a Landsat pixel in a city park next to a pond, there could be 50% urban, 25% vegetation and 25% water in the 30m x 30m pixel.



In [6]:
# Define function to unmix Landsat 5 ee.Image object and add a timestamp
def unmix_L5(image_L5):
    """Unmix each pixel with the given endmembers, by computing the pseudo-inverse
    and multiplying it through each pixel. Returns an image of doubles with the same
    number of bands as endmembers. (Description from GEE docs).

    Parameters
    ----------
    image_L5 : ee.Image
        Any ee.Image object from Landsat 5 (must include at least the first 7 bands).

    Returns
    ------
    unmixed_image : ee.Image
        A 4-band ee.Image object.  Three bands, ('urban', 'veg', and 'water') are the
        per-pixel fraction that the .unmix() method determined that correspond with
        the three given endmember values.  The final band, 'system:time_start' is
        the time metadata associated with that image.
    """
    # Select bands 1-7
    bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
    image = image_L5.select(bands)

    # Endmembers taken from GEE unmix() example script found here:
    # https://developers.google.com/earth-engine/image_transforms
    urban = [88, 42, 48, 38, 86, 115, 59]  # band_0
    veg = [50, 21, 20, 35, 50, 110, 23]  # band_1
    water = [51, 20, 14, 9, 7, 116, 4]  # band_2
    
    # Run the GEE unmix() method with to get per-pixel fractions that are non-negative
    # and sum-to-one
    unmixed_image = image.unmix([urban, veg, water], True, True).rename([
        'urban', 'veg', 'water'])
    
    # Add time band and return image - time stamp is divided by large number to avoid 
    # extra high (or low) scale values.
    return unmixed_image.addBands(image.metadata('system:time_start').divide(1e13))

In [7]:
# Filter Landsat 5 ImageCollection and map the unmix function over the collection
collection = ee.ImageCollection('LANDSAT/LT05/C01/T1').filterDate(
    ee.Date('1995-01-01'), ee.Date('2011-12-31')).filterBounds(roi).map(unmix_L5)

In [8]:
# Reduce the collection with the linear fit reducer - independent variable is
# followed by the dependent variable
linearFit = collection.select(
    ['system:time_start', 'urban']).reduce(ee.Reducer.linearFit())

# Clip the linearFit to the roi
clipped_linearFit = linearFit.clip(roi)

#### Plot 2 Title: Linear Fit, Urban Trend Map from 1995 to 2011

In [9]:
# Initiate second interactive map
Map2 = geemap.Map(center=(39.791424, -104.809968), zoom=10)
Map2.add_basemap('HYBRID')  # Add Google Map

# Display the results
Map2.addLayer(clipped_linearFit,
             {'min': 0, 'max': [-0.9, 8e-5, 1], 'bands': ['scale', 'offset', 'scale']}, 'Linear Fit')

# Add legend to map
legend_keys = ['High Positive Urban Trend', 'Positive Urban Trend',
               'Nearly No Urban Trend', 'Negative Urban Trend']
legend_colors = ['#0000FF', '#01FFFF', '#00FF1F', '#FFFF02']

Map2.add_legend(legend_keys=legend_keys,
               legend_colors=legend_colors, position='bottomleft')

Map2

Map(center=[39.791424, -104.809968], controls=(WidgetControl(options=['position'], widget=HBox(children=(Toggl…

**Plot 2 Caption:** The majority of the plot appears to have a positive urban trend - however this mostly corresponds with rural areas that do not have much development (e.g. east of DIA, the Rocky Mountain Arsenal National Wildlife Refuge).  This makes me wonder if the `scale` band for the Linear Fit output should be interpretted in the inverse - meaning a neative `scale` actually corresponds to a positive urban trend.  However, it is not clear if this is the case.  Curiously, the mathematical interpreation of the results below suggest that yellow corresponds to a positive urban trend (the opposite of what is represented by the `scale` values interpreted on this plot).

In [10]:
out_dir = os.path.join(et.io.HOME, 'Downloads')
filename = os.path.join(out_dir, 'linearFit.tif')


# Exports two files - one file per band ('filename.bandname.tif')
geemap.ee_export_image(clipped_linearFit, filename=filename, scale=90, region=roi, file_per_band=True)

Generating URL ...
Please wait ...
Data downloaded to /Users/richardudell/Downloads


In [11]:
# Confirm that linearFit.scale.tif appears in your Downloads folder
scale_path = os.path.join(et.io.HOME, 'Downloads', 'linearFit.scale.tif')

# Import scale (slope values for each pixel in the ROI)
with rio.open(scale_path) as src:
    scale_tif = src.read()

In [12]:
# Calculate the total number of pixels in the dataset
total = scale_tif < np.inf

# Interpret the slope values as either trending positive or negative (above
# or below zero)
above = scale_tif < 0
below = scale_tif > 0

# Confirm that there are no values at zero
at = scale_tif == 0

print('Number of pixels above zero: ', above.sum(),
      '\nBelow zero: ', below.sum(), 
      '\nAt zero: ', at.sum())

Number of pixels above zero:  21039 
Below zero:  145896 
At zero:  0


In [13]:
# Calculate the total area of the ROI
area_per_pixel = clipped_linearFit.pixelArea()
total_area_roi = area_per_pixel.reduceRegion(ee.Reducer.sum(), roi, 30)
total_area_roi = total_area_roi.getInfo()['area']

In [14]:
# Calculate the percent of pixels trending towards urban
trending_urban_perc = round(above.sum()/total.sum(), 2)

print(trending_urban_perc, '% of the pixels in the ROI are trending towards urban.')
print('This is the same as: ', int(trending_urban_perc * total_area_roi), ' square meters, or', )
print(round((trending_urban_perc * total_area_roi)/2.59e6, 8), ' square miles.')

0.13 % of the pixels in the ROI are trending towards urban.
This is the same as:  104614003  square meters, or
40.39150716  square miles.




# What Did I Find?
Of the 10 mile radius surrounding Denver International Airport, 13% of the land area has a positive urban trend.  The area land that was found to have a positive urban trend is spreadout northwest of the airport and directly north of the Rocky Mountain Arsenal National Wildlife Refuge, as well as directly south and southwest of the airport along Interstate 70.  

# Why Does It Matter? 
significance to general audience - why should they care about the finding?
At 53 sq. miles, DIA is the second largest airport landowner in the world and serves over 58 million passengers annually.  DIA is the largest economic driver in the state, it fuels 26 billion (USD) in annual economic benefit.
([sasaki.com](https://www.sasaki.com/projects/denver-international-airport-strategic-development-plan/))  It is clear that the airport has a large impact on Denver Metro area, as well as the entire state of Colorado.  From this analysis, we now further understand the impact DIA has had on the development of urban land cover most immediately surrounding the airport.


** INCLUDE TWO IMAGES I MADE THAT EXPLAIN SPECTRAL UNMIXING **

# Final Discussion 

i.e. other implications or potential applications of findings, future work
Although it is exciting to quantify the impact that DIA has had on the land cover in the area surrounding the airport, this analysis falls short of providing relevant data.  Future work will include the addition of satellite imagery from Landsat 8 - which has been in operation since 2013.  This will likely require a reconsideration of the endmembers used in the spectral unmixing analysis.  This process is known as endmember extraction and, once learned, would prove to be an excellent skill to have under my belt because of its [versatility and powerful capabilities in remote sensing](https://en.wikipedia.org/wiki/Hyperspectral_imaging).  

# Interested in a Tutorial?
Visit my [GitHub Repository](https://github.com/richardudell/predicting-urban-development) for a complete tutorial on how to do this analysis!
